In [2]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
from parse_tools import *
import pyarrow as pa

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 30)
pd.set_option('compute.use_numexpr', False)

# import warnings
# from pandas.errors import PerformanceWarning
# pd.options.mode.use_inf_as_na = True
# warnings.filterwarnings('ignore', category=PerformanceWarning)

SRC_DIR = './data-indexed'
TGT_DIR = './data-aggregated'

In [4]:
# Load the index from data-indexed
SS_IDX_DIR = os.path.join(SRC_DIR, 'leagues/NBA_2023')
get_all_files(SS_IDX_DIR)

['opp_prev_games_index.csv',
 'opp_prev_h2h_index.csv',
 'opp_prev_opp_index.csv',
 'opp_ven_prev_index.csv',
 'team_opp_games_index.csv',
 'team_prev_games_index.csv',
 'team_prev_h2h_index.csv',
 'team_prev_opp_index.csv',
 'team_ven_prev_index.csv']

In [21]:
SRC_IDX_CSV = os.path.join(SS_IDX_DIR, 'team_ven_prev_index.csv')
IDX_DF = pd.read_csv(SRC_IDX_CSV,index_col=[0,1],header=[0,1])
IDX_DF_TRF = pd.DataFrame(index=IDX_DF.index)
for col in IDX_DF.columns.levels[0]:
    IDX_DF_TRF[col] = list(zip(IDX_DF[col]['index'],IDX_DF[col]['Team_id']))
IDX_DF_TRF

Team_Curr_Gm   
index Team_id                                            
0     /teams/ATL/2023.html   (0, /teams/ATL/2023.html)  \
      /teams/BOS/2023.html   (0, /teams/BOS/2023.html)   
      /teams/BRK/2023.html   (0, /teams/BRK/2023.html)   
      /teams/CHI/2023.html   (0, /teams/CHI/2023.html)   
      /teams/CHO/2023.html   (0, /teams/CHO/2023.html)   
      /teams/CLE/2023.html   (0, /teams/CLE/2023.html)   
      /teams/DAL/2023.html   (0, /teams/DAL/2023.html)   
      /teams/DEN/2023.html   (0, /teams/DEN/2023.html)   
      /teams/DET/2023.html   (0, /teams/DET/2023.html)   
      /teams/GSW/2023.html   (0, /teams/GSW/2023.html)   
      /teams/HOU/2023.html   (0, /teams/HOU/2023.html)   
      /teams/IND/2023.html   (0, /teams/IND/2023.html)   
      /teams/LAC/2023.html   (0, /teams/LAC/2023.html)   
      /teams/LAL/2023.html   (0, /teams/LAL/2023.html)   
      /teams/MEM/2023.html   (0, /teams/MEM/2023.html)   
...                                                ...   
81    /teams/MIA/2023.html  (81, /teams/MIA/2023.html)   
      /teams/MIL/2023.html  (81, /teams/MIL/2023.html)   
      /teams/MIN/2023.html  (81, /teams/MIN/2023.html)   
      /teams/NOP/2023.html  (81, /teams/NOP/2023.html)   
      /teams/NYK/2023.html  (81, /teams/NYK/2023.html)   
      /teams/OKC/2023.html  (81, /teams/OKC/2023.html)   
      /teams/ORL/2023.html  (81, /teams/ORL/2023.html)   
      /teams/PHI/2023.html  (81, /teams/PHI/2023.html)   
      /teams/PHO/2023.html  (81, /teams/PHO/2023.html)   
      /teams/POR/2023.html  (81, /teams/POR/2023.html)   
      /teams/SAC/2023.html  (81, /teams/SAC/2023.html)   
      /teams/SAS/2023.html  (81, /teams/SAS/2023.html)   
      /teams/TOR/2023.html  (81, /teams/TOR/2023.html)   
      /teams/UTA/2023.html  (81, /teams/UTA/2023.html)   
      /teams/WAS/2023.html  (81, /teams/WAS/2023.html)   

                                   Team_Ven_Prev_Gm_01   
index Team_id                                            
0     /teams/ATL/2023.html                    (-1, -1)  \
      /teams/BOS/2023.html                    (-1, -1)   
      /teams/BRK/2023.html                    (-1, -1)   
      /teams/CHI/2023.html                    (-1, -1)   
      /teams/CHO/2023.html                    (-1, -1)   
      /teams/CLE/2023.html                    (-1, -1)   
      /teams/DAL/2023.html                    (-1, -1)   
      /teams/DEN/2023.html                    (-1, -1)   
      /teams/DET/2023.html                    (-1, -1)   
      /teams/GSW/2023.html                    (-1, -1)   
      /teams/HOU/2023.html                    (-1, -1)   
      /teams/IND/2023.html                    (-1, -1)   
      /teams/LAC/2023.html                    (-1, -1)   
      /teams/LAL/2023.html                    (-1, -1)   
      /teams/MEM/2023.html                    (-1, -1)   
...                                                ...   
81    /teams/MIA/2023.html  (77, /teams/MIA/2023.html)   
      /teams/MIL/2023.html  (78, /teams/MIL/2023.html)   
      /teams/MIN/2023.html  (78, /teams/MIN/2023.html)   
      /teams/NOP/2023.html  (76, /teams/NOP/2023.html)   
      /teams/NYK/2023.html  (78, /teams/NYK/2023.html)   
      /teams/OKC/2023.html  (78, /teams/OKC/2023.html)   
      /teams/ORL/2023.html  (80, /teams/ORL/2023.html)   
      /teams/PHI/2023.html  (80, /teams/PHI/2023.html)   
      /teams/PHO/2023.html  (79, /teams/PHO/2023.html)   
      /teams/POR/2023.html  (76, /teams/POR/2023.html)   
      /teams/SAC/2023.html  (79, /teams/SAC/2023.html)   
      /teams/SAS/2023.html  (78, /teams/SAS/2023.html)   
      /teams/TOR/2023.html  (75, /teams/TOR/2023.html)   
      /teams/UTA/2023.html  (77, /teams/UTA/2023.html)   
      /teams/WAS/2023.html  (80, /teams/WAS/2023.html)   

                                   Team_Ven_Prev_Gm_02   
index Team_id                                            
0     /teams/ATL/2023.html                    (-1, -1)  \
      /teams/BOS/2023.html            

In [31]:
ROW = IDX_DF_TRF.iloc[1000]
EMPTY_IDX_VAL = (-1,'-1')
ROW[ROW != EMPTY_IDX_VAL].values

array([(33, '/teams/HOU/2023.html'), (32, '/teams/HOU/2023.html'),
       (24, '/teams/HOU/2023.html'), (22, '/teams/HOU/2023.html'),
       (21, '/teams/HOU/2023.html'), (20, '/teams/HOU/2023.html'),
       (19, '/teams/HOU/2023.html'), (14, '/teams/HOU/2023.html'),
       (12, '/teams/HOU/2023.html'), (11, '/teams/HOU/2023.html'),
       (10, '/teams/HOU/2023.html'), (9, '/teams/HOU/2023.html'),
       (7, '/teams/HOU/2023.html'), (6, '/teams/HOU/2023.html'),
       (5, '/teams/HOU/2023.html'), (4, '/teams/HOU/2023.html'),
       (2, '/teams/HOU/2023.html'), (0, '/teams/HOU/2023.html')],
      dtype=object)

In [39]:
SRC_FACTS_DF = './data-normalized/leagues/NBA_2023/tgl_advanced/stats_expd_avg/raw'
FACTS_DF_LIST = []
for file in get_all_files(SRC_FACTS_DF):
    FACTS_DF_I = pd.read_csv(os.path.join(SRC_FACTS_DF,file),index_col=[0,1],header=[0,1])
    FACTS_DF_LIST.append(FACTS_DF_I)
    
FACTS_DF = pd.concat(FACTS_DF_LIST,axis=0).sort_index()
FACTS_DF

Result                            Advanced   
                              W/L          Tm         Opp        ORtg   
   Team_id                                                              
0  /teams/ATL/2023.html  1.000000  117.000000  107.000000  114.300000  \
   /teams/BOS/2023.html  1.000000  126.000000  117.000000  128.300000   
   /teams/BRK/2023.html  0.000000  108.000000  130.000000  104.800000   
   /teams/CHI/2023.html  1.000000  116.000000  108.000000  112.500000   
   /teams/CHO/2023.html  1.000000  129.000000  102.000000  124.300000   
   /teams/CLE/2023.html  0.000000  105.000000  108.000000  108.700000   
   /teams/DAL/2023.html  0.000000  105.000000  107.000000  109.600000   
   /teams/DEN/2023.html  0.000000  102.000000  123.000000  101.400000   
   /teams/DET/2023.html  1.000000  113.000000  109.000000  112.300000   
   /teams/GSW/2023.html  1.000000  123.000000  109.000000  108.300000   
   /teams/HOU/2023.html  0.000000  107.000000  117.000000  104.500000   
   /teams/IND/2023.html  0.000000  107.000000  114.000000  103.700000   
   /teams/LAC/2023.html  1.000000  103.000000   97.000000   99.100000   
   /teams/LAL/2023.html  0.000000  109.000000  123.000000   96.000000   
   /teams/MEM/2023.html  1.000000  115.000000  112.000000  108.000000   
...                           ...         ...         ...         ...   
81 /teams/MIA/2023.html  0.536585  109.475610  109.792683  113.991463   
   /teams/MIL/2023.html  0.707317  116.939024  113.304878  116.125610   
   /teams/MIN/2023.html  0.512195  115.780488  115.817073  114.517073   
   /teams/NOP/2023.html  0.512195  114.365854  112.475610  115.267073   
   /teams/NYK/2023.html  0.573171  116.024390  113.097561  118.858537   
   /teams/OKC/2023.html  0.487805  117.475610  116.390244  115.921951   
   /teams/ORL/2023.html  0.414634  111.414634  113.975610  112.419512   
   /teams/PHI/2023.html  0.658537  115.219512  110.902439  118.870732   
   /teams/PHO/2023.html  0.548780  113.646341  111.573171  116.056098   
   /teams/POR/2023.html  0.402439  113.402439  117.414634  115.720732   
   /teams/SAC/2023.html  0.585366  120.707317  118.060976  119.967073   
   /teams/SAS/2023.html  0.268293  113.036585  123.073171  111.114634   
   /teams/TOR/2023.html  0.500000  112.853659  111.378049  116.364634   
   /teams/UTA/2023.html  0.451220  117.073171  118.012195  116.414634   
   /teams/WAS/2023.html  0.426829  113.158537  114.365854  115.206098   

                                                                                
                               DRtg        Pace       FTr      3PAr       TS%   
   Team_id                                                                      
0  /teams/ATL/2023.html  104.500000  102.400000  0.267000  0.278000  0.582000  \
   /teams/BOS/2023.html  119.200000   98.200000  0.341000  0.427000  0.668000   
   /teams/BRK/2023.html  126.200000  103.000000  0.270000  0.371000  0.542000   
   /teams/CHI/2023.html  104.700000  103.100000  0.301000  0.387000  0.551000   
   /teams/CHO/2023.html   98.300000  103.800000  0.245000  0.309000  0.619000   
   /teams/CLE/2023.html  111.800000   96.600000  0.299000  0.364000  0.603000   
   /teams/DAL/2023.html  111.700000   95.800000  0.453000  0.467000  0.584000   
   /teams/DEN/2023.html  122.200000  100.600000  0.217000  0.265000  0.561000   
   /teams/DET/2023.html  108.300000  100.600000  0.255000  0.404000  0.540000   
   /teams/GSW/2023.html   96.000000  113.600000  0.232000  0.455000  0.564000   
   /teams/HOU/2023.html  114.300000  102.400000  0.153000  0.357000  0.511000   
   /teams/IND/2023.html  110.500000  103.200000  0.216000  0.433000  0.504000   
   /teams/LAC/2023.html   93.300000  104.000000  0.388000  0.363000  0.550000   
   /teams/LAL/2023.html  108.300000  113.600000  0.266000  0.426000  0.519000   
   /teams/MEM/2023.html  105.200000   96.400000  0.213000  0.463000  0.487000   
...                             ...         ...       ...       ...       ...   
81 /t

In [47]:
QUERY_DF_LIST = []
for I in IDX_DF_TRF.index[2000:2010]:
    ROW_I = IDX_DF_TRF.loc[I]
    ROW_I_IDX_LIST = ROW_I[ROW_I != EMPTY_IDX_VAL].values
    QUERY_DF = FACTS_DF.loc[ROW_I_IDX_LIST]
    QUERY_DF_LIST.append(QUERY_DF)
    # Query all games from the index

QUERY_DF_LIST[0]

Result                            Advanced   
                              W/L          Tm         Opp        ORtg   
   Team_id                                                              
66 /teams/OKC/2023.html  0.477612  118.059701  116.686567  116.123881  \
65 /teams/OKC/2023.html  0.469697  118.181818  117.000000  116.218182   
58 /teams/OKC/2023.html  0.474576  117.864407  116.440678  115.886441   
57 /teams/OKC/2023.html  0.482759  117.913793  116.310345  115.862069   
54 /teams/OKC/2023.html  0.490909  117.945455  116.854545  115.989091   
53 /teams/OKC/2023.html  0.481481  117.574074  116.629630  115.448148   
52 /teams/OKC/2023.html  0.471698  117.283019  116.377358  115.292453   
50 /teams/OKC/2023.html  0.470588  116.647059  115.803922  114.745098   
46 /teams/OKC/2023.html  0.489362  116.574468  115.510638  114.480851   
45 /teams/OKC/2023.html  0.478261  116.913043  115.869565  114.543478   
43 /teams/OKC/2023.html  0.477273  116.795455  116.045455  114.334091   
42 /teams/OKC/2023.html  0.465116  116.906977  116.372093  114.323256   
41 /teams/OKC/2023.html  0.452381  116.738095  116.523810  114.045238   
40 /teams/OKC/2023.html  0.439024  116.341463  116.585366  113.717073   
37 /teams/OKC/2023.html  0.421053  116.105263  117.078947  113.107895   
34 /teams/OKC/2023.html  0.428571  115.742857  116.885714  112.748571   
26 /teams/OKC/2023.html  0.407407  115.777778  117.703704  112.407407   
25 /teams/OKC/2023.html  0.423077  115.846154  117.576923  112.253846   
24 /teams/OKC/2023.html  0.440000  116.400000  117.880000  112.372000   
23 /teams/OKC/2023.html  0.458333  117.000000  117.666667  112.933333   
22 /teams/OKC/2023.html  0.434783  116.826087  117.826087  112.878261   
20 /teams/OKC/2023.html  0.380952  115.857143  117.666667  112.104762   
19 /teams/OKC/2023.html  0.400000  116.600000  118.300000  112.645000   
15 /teams/OKC/2023.html  0.437500  116.187500  116.812500  112.412500   
14 /teams/OKC/2023.html  0.466667  116.600000  116.533333  112.660000   
13 /teams/OKC/2023.html  0.428571  116.285714  116.285714  111.907143   
12 /teams/OKC/2023.html  0.461538  115.846154  115.538462  112.161538   
9  /teams/OKC/2023.html  0.400000  109.700000  111.800000  108.150000   
8  /teams/OKC/2023.html  0.444444  110.444444  111.777778  108.777778   
5  /teams/OKC/2023.html  0.500000  112.333333  111.333333  109.983333   
1  /teams/OKC/2023.html  0.000000  112.500000  118.500000  107.550000   
0  /teams/OKC/2023.html  0.000000  108.000000  115.000000  103.500000   

                                                                                
                               DRtg        Pace       FTr      3PAr       TS%   
   Team_id                                                                      
66 /teams/OKC/2023.html  114.807463  100.617910  0.255179  0.367597  0.574836  \
65 /teams/OKC/2023.html  115.092424  100.625758  0.252985  0.367864  0.574682   
58 /teams/OKC/2023.html  114.513559  100.518644  0.252763  0.360949  0.573169   
57 /teams/OKC/2023.html  114.306897  100.562069  0.252759  0.360276  0.573655   
54 /teams/OKC/2023.html  114.918182  100.605455  0.248709  0.360145  0.575691   
53 /teams/OKC/2023.html  114.531481  100.709259  0.245500  0.361463  0.573370   
52 /teams/OKC/2023.html  114.409434  100.581132  0.246113  0.361453  0.571981   
50 /teams/OKC/2023.html  113.917647  100.478431  0.246588  0.359431  0.569255   
46 /teams/OKC/2023.html  113.468085  100.540426  0.248106  0.363532  0.568596   
45 /teams/OKC/2023.html  113.556522  100.758696  0.249587  0.365087  0.569522   
43 /teams/OKC/2023.html  113.627273  100.784091  0.253114  0.362614  0.568295   
42 /teams/OKC/2023.html  113.837209  100.860465  0.252279  0.363047  0.568674   
41 /teams/OKC/2023.html  113.885714  100.919048  0.252595  0.363143  0.567690   
40 /teams/OKC/2023.html  113.997561  100.836585  0.249390  0.362634  0.565390   
37 /teams/OKC/2023.html  114.134211  101.031579  0.253684  0.363421  0.564184   
34 /teams/OKC

In [48]:
QUERY_DF_LIST[0].mean(axis=0)

Result                  W/L         0.423551
                        Tm        115.805977
                        Opp       116.286744
Advanced                ORtg      112.861335
                        DRtg      113.361849
                        Pace      101.103946
                        FTr         0.237696
                        3PAr        0.368718
                        TS%         0.559415
                        TRB%       48.245554
                        AST%       56.654249
                        STL%        8.048031
                        BLK%       10.000212
Offensive Four Factors  eFG%        0.524431
                        TOV%       11.239153
                        ORB%       24.564741
                        FT/FGA      0.187927
Defensive Four Factors  eFG%        0.535770
                        TOV%       13.980003
                        DRB%       72.946743
                        FT/FGA      0.226214
dtype: float64